# RAP MACHINE  - the generator

## Preprocessing - reading and preparing lyrics data 

In [10]:
import pandas as pd 
#read data from csv file 
data = pd.read_csv('songdata.csv')
#create a set of the 7 rappers to be used...
artists = set(data['artist'])
kanyetext = data.loc[data['artist'] == 'Kanye West','text']
#, 'Drake', 'Mc Hammer', 'Eminem', 'Migos', 'Flo-Rida'
draketext = data.loc[data['artist'] == 'Drake','text']
eminemtext = data.loc[data['artist'] == 'Eminem','text']
migostext = data.loc[data['artist'] == 'Migos', 'text']
floridatext = data.loc[data['artist'] == 'Flo-Rida', 'text']
mchammertext = data.loc[data['artist'] == 'Mc Hammer', 'text']
lilwaynetext = data.loc[data['artist'] == 'Lil Wayne', 'text']

text = [kanyetext, draketext, eminemtext, migostext, mchammertext, floridatext, lilwaynetext]
text = pd.concat(text)
print(len(kanyetext))
print(len(text))

106
510


In [14]:
import string

text = ''.join(text) 
#builds a translation table 
text = text.lower().translate(str.maketrans('','', ".!()-\""))
print(text[:1000])

why are you so paranoid?  
don't be so paranoid  
don't be so  
  
baby, don't worry about it  
hey there, don't even think about it  
  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
  
all of the time, you really wanna spend your whole life alone  
a little time out might do ya good, might do us good before you're done for
good  
because i could make it good, i could make it hood, i could make you come, i
could make you gooo  
i could make you hot, i could make you fly, make you touch the sky, hey, maybe
sooo  
all of the time, he be up in my, checking through my cell phone, baby no  
you wanna kill the vibe, on another night, here's another fight, oh, here we
go oh, here we go  
  
baby, don't worry 'bout it  
lady, we'll go out to the floor  
  
anyway, they don't know you like i do they don't know  
they'll never know you

## Processing 

In [12]:
#LSTM implementation adapted from https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys


chars = sorted(list(set(text))) #get all unique characters in the text
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) #map character to index
indices_char = dict((i, c) for i, c in enumerate(chars)) #map index to character



total chars: 44


### word by word...

In [13]:
import string

text = [kanyetext, draketext, eminemtext, migostext, lilwaynetext]
songs = []
#create translation table that maps out words like a dictionary
for artist in text:
    for i in artist:
        songs.append(i.lower().translate(str.maketrans('','', ".!()-\""))) #remove these characters from the text
        
text = songs
print(text[0])

why are you so paranoid?  
don't be so paranoid  
don't be so  
  
baby, don't worry about it  
hey there, don't even think about it  
  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
you worry bout the wrong things, the wrong things  
  
all of the time, you really wanna spend your whole life alone  
a little time out might do ya good, might do us good before you're done for
good  
because i could make it good, i could make it hood, i could make you come, i
could make you gooo  
i could make you hot, i could make you fly, make you touch the sky, hey, maybe
sooo  
all of the time, he be up in my, checking through my cell phone, baby no  
you wanna kill the vibe, on another night, here's another fight, oh, here we
go oh, here we go  
  
baby, don't worry 'bout it  
lady, we'll go out to the floor  
  
anyway, they don't know you like i do they don't know  
they'll never know you

### Getting ready to train the model by collecting last bits of data into lists

In [42]:
# cut the text in semi-redundant sequences of maxlen characters, split by song

maxlen = 40
step = 3
sentences = []
next_chars = []

#breaks text up like this:     
'''["hello darkness my old friend i've come t", 
    "lo darkness my old friend i've come to t", 
    "darkness my old friend i've come to talk", 
    "kness my old friend i've come to talk wi", 
    "ss my old friend i've come to talk with ", 
    "my old friend i've come to talk with you", 
    "old friend i've come to talk with you ag"] --> sentences, and then puts the next char in next_chars'''

for i in range(len(text)): 
    for j in range(0, len(text[i]) - maxlen, step):
        sentences.append(text[i][j: j + maxlen])
        next_chars.append(text[i][j + maxlen])
print('number of sequences:', len(sentences))

number of sequences: 356060


In [43]:
print('Vectorization...')

#initialize two arrays, X = (maxlen rows x len(chars) cols) x len(sentences), y = len(sentences x len(chars))
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

#for every sequence, mark the appearance of every char in that sequence and then mark the corresponding next char for that sequence
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


### Training the model!

In [ ]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(len(chars))) 
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    random_index = random.randint(0, len(text[start_index]) - maxlen - 1) #another random seed
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index][random_index: random_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
356060/356060 [==============================] - 1504s - loss: 2.0209  

----- diversity: 0.2
----- Generating with seed: "on, fuck your fresh  
head shots nigga f"
on, fuck your fresh  
head shots nigga for the  
  
i got to the lot the bat you  
  
what i would be to the plant to  
i want the gand to to streally  
so me  
i want to the plan  
i want the never my back  
i want to the hat the be  
i wond the baby  
i want to the be  
i'm a wast to baby  
i got to the bot the be to the bot  
i got the baby  
i want the streak to the wast  
i got to the bad  
you ain't was the neal  
they can't to a 

----- diversity: 0.5
----- Generating with seed: "on, fuck your fresh  
head shots nigga f"
on, fuck your fresh  
head shots nigga for the bat we got to money  
man a want  
  
make bab  
i is i get spest  
they say my heald right it  
with come  
work to me  
  
when i works up in the neak to down be  

/home/lebronjames123/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


l the be  
i still the still the be the cans  
i ain't got the stop the girl  
  
i'm stand  
  
i got the the be it  
i got the that i think the chow  
i got the can the got the get the was the the be  
the got the the shit the let me  
i got the the think  
  
i got the man  
i can't the be the still it and i ain't stand  
i was the go the was we was 

----- diversity: 0.5
----- Generating with seed: "ut of mind  
but if i am it's just in ti"
ut of mind  
but if i am it's just in time  
and that i got a gring botter  
but i'm say  
  
off  
  
i'm got me i thing  
you got it to don't got to me  
and i'm don't was slick  
i got a bes  
i can the say the got it do the de't  
i got a bitch you that the know the nal  
i lever hat your when i thing  
they have you got take my bout be  
i still a down i said give it at you the gicker  
i was hap niggas starters that supper the be 

----- diversity: 1.0
----- Generating with seed: "ut of mind  
but if i am it's just in ti"
ut of mind  
but 